### Chapter6_Queation 8

In this exercise, we will generate simulated data, and will then use this data to perform forward and backward stepwise selection.

(a) Create a random number generator and use its normal() method to generate a predictor $X$ of length n = 100, as well as a noise
vector $\epsilon$ of length n = 100.

(b) Generate a response vector $Y$ of length n = 100 according to the model:

$
Y= β_{0} + β_{1}X + β_{2}X_{2} + β_{3}X_{3} + \epsilon,
$

where $β_{0}$, $β_{1}$, $β_{2}$ and $β_{3}$ are constants of your choice.

(c) Use forward stepwise selection in order to select a model containing the predictors $X$, $X^2$, . . . , $X^{10}$. What is the model obtained according to $C_{p}$? Report the coefficients of the model obtained.

(d) Repeat (c), using backwards stepwise selection. How does your answer compare to the results in (c)?

(e) Now fit a lasso model to the simulated data, again using $X$, $X_{2}$, ... , $X_{10}$ as predictors. Use cross-validation to select the optimal value of $\lambda$. Create plots of the cross-validation error as a function of $\lambda$. Report the resulting coefficient estimates, and discuss the results obtained.

(f) Now generate a response vector $Y$ according to the model

$
Y= \beta_{0} + \beta_{7}X^{7} + \epsilon,
$

and perform forward stepwise selection and the lasso. Discuss the results obtained.